In [4]:
import os
import pandas as pd
from translate import translate
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image

# -----------------------------
# 1️⃣ Build DataFrame with translated labels
# -----------------------------
dataset_path = r"C:\Users\rache\Downloads\archive\raw-img"
data = []

# Iterate through each folder in the dataset directory
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)

    # Check if the current path is actually a directory
    if os.path.isdir(folder_path):
        # Get the label from the 'translate' dict, or default to folder name
        label = translate.get(folder, folder)

        # Iterate through files inside the category folder
        for file in os.listdir(folder_path):
            # Filter specifically for image formats
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(folder_path, file)
                # Store the full path and its corresponding label
                data.append([img_path, label])

# Create a Pandas DataFrame to manage metadata
df = pd.DataFrame(data, columns=["image_path", "label"])

# Display the first few rows and distribution stats
print(df.head())
print("Total images:", len(df))
print(df['label'].value_counts())

# -----------------------------
# 2️⃣ Train / Test Split (90/10)
# -----------------------------
train_df, test_df = train_test_split(
    df,
    test_size=0.1,
    stratify=df['label'],
    random_state=42
)
print("Train size:", len(train_df))
print("Test size:", len(test_df))

# -----------------------------
# 3️⃣ Function to convert DataFrame to tf.data.Dataset
# -----------------------------
IMG_SIZE = 224
BATCH_SIZE = 32

def df_to_dataset(df, shuffle=True, batch_size=32):
    # Extract file paths and convert labels from strings to integer codes
    paths = df['image_path'].values
    labels = df['label'].astype('category').cat.codes.values  # string -> integer

    # Create a basic TensorSliceDataset
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    # Internal function to load and preprocess images
    def process_path(path, label):
        # Read the raw file from disk
        img = tf.io.read_file(path)
        # Decode the JPEG format into a tensor (0-255 integers)
        img = tf.image.decode_jpeg(img, channels=3)
        # Resize image to the input size required by the model (224x224)
        img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
        # Normalize pixel values to range [0, 1] (crucial for convergence)
        img = img / 255.0
        return img, label

    # Apply the processing function in parallel using available CPU cores
    ds = ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

    # Shuffle the dataset to prevent the model from learning order-based patterns
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))

    # Group examples into batches
    ds = ds.batch(batch_size)
    # Prefetch data to GPU memory while CPU prepares the next batch (pipeline optimization)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    return ds

# -----------------------------
# 4️⃣ Create TensorFlow Datasets
# -----------------------------
train_dataset = df_to_dataset(train_df, batch_size=BATCH_SIZE)   # shuffled for training
test_dataset = df_to_dataset(test_df, shuffle=False, batch_size=BATCH_SIZE)  # no shuffle for testing

# -----------------------------
# 5️⃣ Quick verification
# -----------------------------
for images, labels in train_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    print("Example labels:", labels[:10].numpy())
    print("Min pixel:", images.numpy().min())
    print("Max pixel:", images.numpy().max())

# Visualize the first image
img = images[0].numpy()
Image.fromarray((img * 255).astype("uint8")).show()


                                          image_path label
0  C:\Users\rache\Downloads\archive\raw-img\cane\...   dog
1  C:\Users\rache\Downloads\archive\raw-img\cane\...   dog
2  C:\Users\rache\Downloads\archive\raw-img\cane\...   dog
3  C:\Users\rache\Downloads\archive\raw-img\cane\...   dog
4  C:\Users\rache\Downloads\archive\raw-img\cane\...   dog
Total images: 26179
label
dog          4863
ragno        4821
chicken      3098
horse        2623
butterfly    2112
cow          1866
squirrel     1862
sheep        1820
cat          1668
elephant     1446
Name: count, dtype: int64
Train size: 23561
Test size: 2618
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32,)
Example labels: [3 7 5 2 6 7 2 7 0 7]
Min pixel: 0.0
Max pixel: 1.0


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image
import numpy as np

# -----------------------------
# 1️⃣ Déterminer le nombre de classes
# -----------------------------
num_classes = len(df['label'].unique())
print("Number of classes:", num_classes)

# -----------------------------
# 2️⃣ Définir le CNN from scratch
# -----------------------------
model = tf.keras.Sequential([

    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(96, 3, activation='relu'),   # au lieu de 128
    layers.MaxPooling2D(),

    layers.GlobalAveragePooling2D(),  # remplace Flatten (énorme diff)

    layers.Dense(64, activation='relu'),  # au lieu de 128
    layers.Dropout(0.4),

    layers.Dense(num_classes, activation='softmax')
])


# -----------------------------
# 3️⃣ Compiler le modèle
# -----------------------------
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# 4️⃣ Entraîner le modèle
# -----------------------------
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=15   # tu peux augmenter si tu as le temps et le GPU
)

# -----------------------------
# 5️⃣ Tester / visualiser quelques prédictions
# -----------------------------
# Mapping entier -> label
label_mapping = dict(enumerate(df['label'].astype('category').cat.categories))

# Prédire sur un batch de test
for images, labels in test_dataset.take(1):
    preds = model.predict(images)
    pred_classes = np.argmax(preds, axis=1)

    for i in range(len(images)):
        img = images[i].numpy()
        Image.fromarray((img*255).astype("uint8")).show(title=label_mapping[pred_classes[i]])
        print("Predicted:", label_mapping[pred_classes[i]], "| Actual:", label_mapping[labels[i].numpy()])
        # done 


Number of classes: 10


c:\Users\rache\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 96)     │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 96)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 81,642 (318.91 KB)

 Trainable params: 81,642 (318.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
737/737 ━━━━━━━━━━━━━━━━━━━━ 225s 278ms/step - accuracy: 0.2270 - loss: 2.1247 - val_accuracy: 0.2865 - val_loss: 1.9676
Epoch 2/15
737/737 ━━━━━━━━━━━━━━━━━━━━ 219s 276ms/step - accuracy: 0.3201 - loss: 1.9146 - val_accuracy: 0.3594 - val_loss: 1.7681
Epoch 3/15
737/737 ━━━━━━━━━━━━━━━━━━━━ 218s 273ms/step - accuracy: 0.3751 - loss: 1.7755 - val_accuracy: 0.4335 - val_loss: 1.6070
Epoch 4/15


In [2]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))


Num GPUs Available: 0
[]
